<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [14]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

In [1]:
import sys
import numpy as np

def prepare_to_shakespeare(url, number_of_chars):
    
    import urllib.request
    
    req = urllib.request.Request(url)
    with urllib.request.urlopen(req) as response:
        data = response.read().decode("utf8")[:number_of_chars]

    data_stripped = [item.strip("\ufeff|\x80|\x99|\n|\r|ï|¿|#|»|9|8|0|â|ï") for item in data] 

    return data_stripped

In [2]:
# Apply our function
works = prepare_to_shakespeare('https://www.gutenberg.org/files/100/100-0.txt', 100000)

In [3]:
type(works)

list

In [5]:
# Turn works into a long string
text = ''.join(works)

In [9]:
type(text), text[:1000]

(str,
 'Project Gutenberg’s The Complete Works of William Shakespeare, by WilliamShakespeareThis eBook is for the use of anyone anywhere in the United States andmost other parts of the world at no cost and with almost no restrictionswhatsoever.  You may copy it, give it away or re-use it under the termsof the Project Gutenberg License included with this eBook or online atwww.gutenberg.org.  If you are not located in the United States, you’llhave to check the laws of the country where you are located before usingthis ebook.Title: The Complete Works of William ShakespeareAuthor: William ShakespeareRelease Date: January 14 [EBook 1]Last Updated: November 7, 21Language: EnglishCharacter set encoding: UTF-*** START OF THIS PROJECT GUTENBERG EBOOK THE COMPLETE WORKS OF WILLIAM SHAKESPEARE ***The Complete Works of William Shakespeareby William Shakespeare      Contents               THE SONNETS               ALL’S WELL THAT ENDS WELL               THE TRAGEDY OF ANTONY AND CLEOPATRA          

In [11]:
''' text is a long string. We just want the unique characters
as a list'''
unique_chars = list(set(text))
print(unique_chars)

['U', '4', 'a', 'O', 'P', 'y', 'q', '(', 'J', 'h', 'm', '!', 'e', 'N', 'Y', 'I', 'V', 'z', 'H', 'r', 'C', '‘', 'x', ')', 'M', 'c', 'X', 'd', 'E', '*', 'B', 'k', '[', 'l', '5', 'n', 's', 'i', 'p', ';', 'W', 'K', 'j', 'g', 'f', '7', 'v', 'D', ']', 'u', ':', '-', 'G', '.', 'A', 'F', '6', 'w', '1', ',', 'o', 'b', 'S', 'T', '3', 't', 'R', '2', '’', ' ', 'L', '?']


In [12]:
# Assign each unique character to a number. Easy way is by enumerating
char_int = {c:i for i,c in enumerate(unique_chars)}
int_char = {i:c for i,c in enumerate(unique_chars)}

In [13]:
## Create Sequence Data

# Scan text variable by 40 character chunks to create first sequence
maxlen = 40
step = 1

# Encode as sequence data as its unique character
encoded = [char_int[unique_chars] for unique_chars in text]

batch_size = 32

sequences = [] # Each element is 40 characters long
next_chars = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i : i + maxlen])
    next_chars.append(encoded[i + maxlen])
print('sequences:', len(sequences))

sequences: 94288


In [15]:
# Specify x & y
# you can onehotencode
# rows, width, dimensions are vector of 40 characters
x = np.zeros((len(sequences), maxlen, len(unique_chars)), dtype=np.bool)
# rows, width (1)
y = np.zeros((len(sequences), len(unique_chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_chars[i]] = 1

x.shape, y.shape

((94288, 40, 72), (94288, 72))

In [21]:
# build the model: a single LSTM
model = Sequential()
# the input_shape of a singular observation
# 40 chars, so multiclass, therefore softmax
model.add(LSTM(128, input_shape=(maxlen, len(unique_chars))))
model.add(Dense(len(unique_chars), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [22]:
#return index location of most probable character
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [23]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = np.random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(unique_chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_int[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = int_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [25]:
model.fit(x, y,
          batch_size=128,
          epochs=5,
          callbacks=[print_callback])

Epoch 1/5
94208/94288 [============================>.] - ETA: 0s - loss: 2.2760
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "s rage shall beauty hold a plea,Whose ac"
s rage shall beauty hold a plea,Whose act in the the the the the the sor the the the shes the the seat the the the sath the the the the the the the the the the sere the the the the the the the thou the the the the the the the the the the sore the the that the the the the the that the the the the the the the the the sher the seath the seath the the the the the the the the so the the the the the the the the the the the the the the the the
----- diversity: 0.5
----- Generating with seed: "s rage shall beauty hold a plea,Whose ac"
s rage shall beauty hold a plea,Whose acl an that the the gith the the pris the psill seind an mund hout thy whee the theperes shat be be thas thee sout that on with thou prot llome the the  ale the beats to he the  in the the thou pall or thet wome and I pr